In [7]:
import os
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("CricketPrediction").getOrCreate()


# Specify the directory where your CSV files are located
directory = r'D:\github\Cricket-Prediction\data\rawData'
battingdata = spark.read.csv(os.path.join(directory, 't20_batting_stats.csv'), header=True, inferSchema=True)
battingdata.show(5)

+--------------------+---+----+---+----+---+----+---+-----+---+---+---+---+---+-------+
|              Player|Mat|Inns| NO|Runs| HS| Ave| BF|   SR|100| 50|  0| 4s| 6s| Season|
+--------------------+---+----+---+----+---+----+---+-----+---+---+---+---+---+-------+
|Zulqarnain Haider...|  2|   2|  0|  18| 17|9.00| 21|85.71|  0|  0|  0|  1|  0|2010/11|
|Zulqarnain Haider...|  1|   1|  0|   5|  5|5.00|  9|55.55|  0|  0|  0|  0|  0|2006/07|
|Zulqarnain Haider...|  1|   -|  -|   -|  -|   -|  -|    -|  -|  -|  -|  -|  -|2019/20|
|Zulqarnain Haider...|  4|   1|  0|   0|  0|0.00|  0|    -|  0|  0|  1|  0|  0|   2019|
|Zulqarnain Haider...|  6|   3|  1|   8|  4|4.00| 12|66.66|  0|  0|  0|  0|  0|   2022|
+--------------------+---+----+---+----+---+----+---+-----+---+---+---+---+---+-------+
only showing top 5 rows

